In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
track2=track3[(track3["GP"]=='Italian Grand Prix')&(track3["Year"]==2024)]
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [5]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_2371/4173085553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2['LapTime']= pd.to_timedelta(track2["LapTime"])


In [6]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]

In [7]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
20555,0 days 00:58:43.525000,PIA,81,0 days 00:01:25.116000,2.0,1.0,NaT,NaT,0 days 00:00:27.751000,0 days 00:00:29.311000,0 days 00:00:28.054000,0 days 00:57:46.160000,0 days 00:58:15.471000,0 days 00:58:43.525000,315.0,312.0,309.0,320.0,True,MEDIUM,2.0,True,McLaren,0 days 00:57:18.409000,NaT,1,1.0,False,,False,True,2024,Italian Grand Prix
20556,0 days 00:58:44.361000,LEC,16,0 days 00:01:25.396000,2.0,1.0,NaT,NaT,0 days 00:00:27.707000,0 days 00:00:29.265000,0 days 00:00:28.424000,0 days 00:57:46.661000,0 days 00:58:15.926000,0 days 00:58:44.350000,315.0,325.0,312.0,325.0,True,MEDIUM,2.0,True,Ferrari,0 days 00:57:18.965000,NaT,1,2.0,False,,False,True,2024,Italian Grand Prix
20558,0 days 00:58:45.215000,NOR,4,0 days 00:01:25.458000,2.0,1.0,NaT,NaT,0 days 00:00:27.676000,0 days 00:00:29.408000,0 days 00:00:28.374000,0 days 00:57:47.443000,0 days 00:58:16.851000,0 days 00:58:45.225000,309.0,332.0,313.0,339.0,True,MEDIUM,2.0,True,McLaren,0 days 00:57:19.757000,NaT,1,3.0,False,,False,True,2024,Italian Grand Prix
20559,0 days 00:58:45.862000,SAI,55,0 days 00:01:25.731000,2.0,1.0,NaT,NaT,0 days 00:00:27.553000,0 days 00:00:29.775000,0 days 00:00:28.403000,0 days 00:57:47.672000,0 days 00:58:17.447000,0 days 00:58:45.850000,318.0,333.0,313.0,331.0,True,MEDIUM,2.0,True,Ferrari,0 days 00:57:20.131000,NaT,1,4.0,False,,False,True,2024,Italian Grand Prix
20561,0 days 00:58:46.503000,HAM,44,0 days 00:01:26.043000,2.0,1.0,NaT,NaT,0 days 00:00:27.701000,0 days 00:00:29.754000,0 days 00:00:28.588000,0 days 00:57:48.128000,0 days 00:58:17.882000,0 days 00:58:46.470000,319.0,337.0,311.0,331.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:57:20.460000,NaT,1,5.0,False,,False,True,2024,Italian Grand Prix


In [8]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) tp slower
team_order = (
    transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['PIA', 'NOR', 'VER', 'HAM', 'LEC', 'SAI', 'RUS', 'PER', 'MAG', 'ALO',
       'STR', 'COL', 'RIC', 'ALB', 'HUL', 'GAS', 'ZHO', 'OCO', 'BOT', 'TSU'],
      dtype='object', name='Driver')


In [9]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
513177,0 days 02:11:29.815000,MAG,20,0 days 00:01:23.715000,53.0,2.0,NaT,NaT,0 days 00:00:27.488000,0 days 00:00:28.468000,0 days 00:00:27.759000,0 days 02:10:33.587000,0 days 02:11:02.055000,0 days 02:11:29.814000,317.0,328.0,311.0,323.0,False,HARD,39.0,True,Haas F1 Team,0 days 02:10:06.100000,NaT,1,9.0,False,,False,True,2024,Italian Grand Prix,83.715,84.793
513186,0 days 02:11:38.969000,ALB,23,0 days 00:01:23.918000,53.0,2.0,NaT,NaT,0 days 00:00:27.501000,0 days 00:00:28.675000,0 days 00:00:27.742000,0 days 02:10:42.534000,0 days 02:11:11.209000,0 days 02:11:38.951000,318.0,326.0,310.0,336.0,True,HARD,36.0,True,Williams,0 days 02:10:15.051000,NaT,1,10.0,False,,False,True,2024,Italian Grand Prix,83.918,85.059
513188,0 days 02:11:40.008000,ALO,14,0 days 00:01:23.686000,53.0,3.0,NaT,NaT,0 days 00:00:27.435000,0 days 00:00:28.394000,0 days 00:00:27.857000,0 days 02:10:43.773000,0 days 02:11:12.167000,0 days 02:11:40.024000,320.0,319.0,314.0,328.0,False,HARD,20.0,False,Aston Martin,0 days 02:10:16.322000,NaT,1,11.0,False,,False,True,2024,Italian Grand Prix,83.686,84.882
513190,0 days 02:11:49.965000,RIC,3,0 days 00:01:24.347000,53.0,2.0,NaT,NaT,0 days 00:00:27.486000,0 days 00:00:28.661000,0 days 00:00:28.200000,0 days 02:10:53.090000,0 days 02:11:21.751000,0 days 02:11:49.951000,316.0,320.0,312.0,321.0,False,HARD,43.0,False,RB,0 days 02:10:25.618000,NaT,1,12.0,False,,False,True,2024,Italian Grand Prix,84.347,85.033
513193,0 days 02:11:52.821000,COL,43,0 days 00:01:23.728000,53.0,2.0,NaT,NaT,0 days 00:00:27.470000,0 days 00:00:28.560000,0 days 00:00:27.698000,0 days 02:10:56.547000,0 days 02:11:25.107000,0 days 02:11:52.805000,317.0,329.0,315.0,338.0,True,HARD,37.0,True,Williams,0 days 02:10:29.093000,NaT,1,13.0,False,,False,True,2024,Italian Grand Prix,83.728,84.939


In [10]:
fig_pace=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Driver",
    y="LapTime (s)",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace Plot for the {} {}</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig_pace.update_traces(opacity=1)
fig_pace.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=50, r=5, t=35, b=50)
)
fig_pace.show()

In [11]:
fig_pace.write_html("/Users/rdesh723/statpulse-html/plots/Race Driver Pace/Driver Pace Plots/{}/{} {} Driver Pace Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [12]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
20555,PIA,0.0000,100.0%
20558,NOR,0.1230,100.147%
20563,VER,0.1515,100.182%
20561,HAM,0.1885,100.226%
20556,LEC,0.2480,100.297%
20559,SAI,0.4580,100.549%
20566,RUS,0.8120,100.973%
20567,PER,1.0560,101.265%
31362,MAG,1.3250,101.587%
31357,ALO,1.4140,101.694%


In [13]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
20555,PIA,0.0010,100.0%
20558,NOR,0.1230,100.147%
20563,VER,0.1515,100.182%
20561,HAM,0.1885,100.226%
20556,LEC,0.2480,100.297%
20559,SAI,0.4580,100.549%
20566,RUS,0.8120,100.973%
20567,PER,1.0560,101.265%
31362,MAG,1.3250,101.587%
31357,ALO,1.4140,101.694%


In [14]:
fig_median=px.bar(
    transformed_laps_relative_median,
    x="Driver",
    y="Delta",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Median Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent",
    height=700, 
    width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig_median.update_traces(textposition='outside')

fig_median.update_traces(opacity=0.85)

fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

fig_median.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_median.update_xaxes(title_text='')      # Remove axis titles
fig_median.update_yaxes(title_text='')

fig_median.update_xaxes(
        title_standoff = 75
)

fig_median.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
fig_median.show()

In [15]:
fig_median.write_html("/Users/rdesh723/statpulse-html/plots/Race Driver Pace/Median Lap Times/{}/{} {} Driver Median Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [16]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
20558,NOR,0.000,100.0%
20561,HAM,0.080,100.098%
20563,VER,0.313,100.384%
20555,PIA,0.511,100.628%
20566,RUS,0.604,100.742%
31378,STR,0.800,100.982%
31357,ALO,1.512,101.857%
20567,PER,1.539,101.89%
20559,SAI,1.787,102.194%
20556,LEC,1.794,102.203%


In [17]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
20558,NOR,0.001,100.0%
20561,HAM,0.080,100.098%
20563,VER,0.313,100.384%
20555,PIA,0.511,100.628%
20566,RUS,0.604,100.742%
31378,STR,0.800,100.982%
31357,ALO,1.512,101.857%
20567,PER,1.539,101.89%
20559,SAI,1.787,102.194%
20556,LEC,1.794,102.203%


In [18]:
fig_min=px.bar(
    transformed_laps_relative_min_all,
    x="Driver",
    y="Delta_Min_All",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig_min.update_traces(textposition='outside')

fig_min.update_traces(opacity=0.85)

fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

fig_min.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_min.update_xaxes(title_text='')      # Remove axis titles
fig_min.update_yaxes(title_text='')

fig_min.update_xaxes(
        title_standoff = 75
)

fig_min.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_min.update_yaxes(ticksuffix = "  ")

fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_min.show()

In [19]:
fig_min.write_html("/Users/rdesh723/statpulse-html/plots/Race Driver Pace/Fastest Lap/{}/{} {} Driver Fastest Lap Plot.html".format(year, year,gp),full_html=False, include_plotlyjs='cdn')

In [20]:
#Multi Race Plot

In [12]:
#2024
race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2024
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    track2['LapTime']= pd.to_timedelta(track2["LapTime"])
    track2=track2[track2['LapNumber']!=1.0]
    quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
    transformed_laps = quicklaps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

    # order the team from the fastest (lowest median lap time) tp slower
    team_order = (
        transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
    )
    print(team_order)
    transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
    transformed_laps.tail()
    fig_pace=px.box(
        transformed_laps.sort_values(by=["median","LapNumber"]),
        x="Driver",
        y="LapTime (s)",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace Plot for the {} {}</b>".format(year_multi,i),
        height=700, width=1200,

    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            }
    )
    fig_pace.update_traces(opacity=1)
    fig_pace.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow",
        margin=dict(l=50, r=5, t=35, b=50)
    )
    fig_pace.write_html("/Users/rdesh723/statpulse-html/plots/Race Driver Pace/Driver Pace Plots/{}/{} {} Driver Pace Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')
    #Get MINIMUM OF THE MEDIAN LAP TIMES

    fastest_lap = transformed_laps["median"].min()
    transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
    transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
    transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
    transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
    transformed_laps_relative_median
    for index, row in transformed_laps_relative_median.iterrows():
        if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
         transformed_laps_relative_median.loc[index, 'Delta']=0.001
    transformed_laps_relative_median
    fig_median=px.bar(
        transformed_laps_relative_median,
        x="Driver",
        y="Delta",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace (Median Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent",
        height=700, 
        width=1200,
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            }
    )

    fig_median.update_traces(textposition='outside')

    fig_median.update_traces(opacity=0.85)

    fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

    fig_median.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_median.update_xaxes(title_text='')      # Remove axis titles
    fig_median.update_yaxes(title_text='')

    fig_median.update_xaxes(
            title_standoff = 75
    )

    fig_median.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

    fig_median.write_html("/Users/rdesh723/statpulse-html/plots/Race Driver Pace/Median Lap Times/{}/{} {} Driver Median Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

    transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

    fastest_lap_all = transformed_laps["min_all"].min()
    transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
    transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
    transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
    transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
    transformed_laps_relative_min_all
    for index, row in transformed_laps_relative_min_all.iterrows():
        if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
         transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
    transformed_laps_relative_min_all
    fig_min=px.bar(
        transformed_laps_relative_min_all,
        x="Driver",
        y="Delta_Min_All",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace (Fastest Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent_All",
        height=700, width=1200,
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            }
    )

    fig_min.update_traces(textposition='outside')

    fig_min.update_traces(opacity=0.85)

    fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

    fig_min.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_min.update_xaxes(title_text='')      # Remove axis titles
    fig_min.update_yaxes(title_text='')

    fig_min.update_xaxes(
            title_standoff = 75
    )

    fig_min.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_min.update_yaxes(ticksuffix = "  ")

    fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_min.write_html("/Users/rdesh723/statpulse-html/plots/Race Driver Pace/Fastest Lap/{}/{} {} Driver Fastest Lap Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['VER', 'SAI', 'PER', 'RUS', 'NOR', 'HAM', 'PIA', 'LEC', 'SAR', 'HUL',
       'STR', 'ALO', 'ZHO', 'GAS', 'TSU', 'MAG', 'RIC', 'OCO', 'BOT', 'ALB'],
      dtype='object', name='Driver')
Index(['VER', 'PER', 'LEC', 'NOR', 'PIA', 'HAM', 'RUS', 'ALO', 'BEA', 'HUL',
       'ZHO', 'MAG', 'ALB', 'OCO', 'SAR', 'TSU', 'BOT', 'RIC', 'STR'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['SAI', 'NOR', 'LEC', 'PER', 'PIA', 'RUS', 'ALO', 'HUL', 'BOT', 'OCO',
       'STR', 'TSU', 'ALB', 'RIC', 'ZHO', 'GAS', 'MAG', 'HAM', 'VER'],
      dtype='object', name='Driver')
Index(['NOR', 'VER', 'LEC', 'PER', 'HAM', 'RUS', 'SAI', 'PIA', 'ALO', 'TSU',
       'STR', 'SAR', 'OCO', 'BOT', 'MAG', 'HUL', 'GAS', 'ZHO'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['VER', 'RUS', 'NOR', 'SAI', 'LEC', 'PIA', 'HAM', 'PER', 'HUL', 'GAS',
       'MAG', 'ZHO', 'RIC', 'OCO', 'TSU', 'STR', 'ALO', 'ALB', 'BOT', 'SAR'],
      dtype='object', name='Driver')
Index(['HAM', 'NOR', 'RUS', 'PIA', 'VER', 'SAI', 'LEC', 'STR', 'HUL', 'ALO',
       'ZHO', 'PER', 'TSU', 'ALB', 'MAG', 'SAR', 'RIC', 'OCO', 'BOT'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['RUS', 'LEC', 'PER', 'HAM', 'PIA', 'NOR', 'VER', 'SAI', 'ALO', 'RIC',
       'HUL', 'STR', 'TSU', 'OCO', 'ALB', 'SAR', 'ZHO', 'MAG', 'BOT', 'GAS'],
      dtype='object', name='Driver')
Index(['HAM', 'RUS', 'LEC', 'NOR', 'PIA', 'VER', 'PER', 'SAI', 'RIC', 'SAR',
       'ALB', 'ALO', 'TSU', 'OCO', 'MAG', 'STR', 'GAS', 'BOT', 'HUL'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7730/647462274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Driver","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [201]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [1]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Driver",
#     y="Delta_Avg_All",
#     color='Driver',
#     template="xgridoff",
#     title="<b>Driver Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
#     color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[3.5, 0])

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=12,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
# fig.show()

In [166]:
# fig.write_html("{} {} Driver Avg Lap Plot.html".format(year,gp))